# Universal Format (UF)

In [ ]:
import cmweather  # noqa
import xarray as xr
from open_radar_data import DATASETS

import xradar as xd

## Download

Fetching Universal Format radar data file from [open-radar-data](https://github.com/openradar/open-radar-data) repository.

In [ ]:
import atexit
from pathlib import Path
from tempfile import TemporaryDirectory

tmpdir_obj = TemporaryDirectory()
atexit.register(tmpdir_obj.cleanup)  # remove even if you forget
tmpdir = Path(tmpdir_obj.name)


def get_temp_file(fname):
    import gzip
    import shutil

    fnamei = Path(DATASETS.fetch(fname))
    fnameo = tmpdir / fnamei.stem
    with gzip.open(fnamei) as fin:
        with open(fnameo, "wb") as fout:
            shutil.copyfileobj(fin, fout)
    return fnameo


fname = get_temp_file("20110427_164233_rvp8-rel_v001_SUR.uf.gz")

## xr.open_dataset

Making use of the xarray `uf` backend. We also need to provide the group. Note, that we are using CfRadial2 group access pattern.

In [ ]:
ds = xr.open_dataset(fname, group="sweep_0", engine="uf")
display(ds)

In [ ]:
import numpy as np

np.testing.assert_almost_equal(ds.sweep_fixed_angle.values, 0.703125)

### Plot Time vs. Azimuth

In [ ]:
ds.azimuth.plot()

### Plot Range vs. Time

We need to sort by time and specify the y-coordinate.

In [ ]:
ds.DBZH.sortby("time").plot(y="time", cmap="HomeyerRainbow")

### Plot Range vs. Azimuth


In [ ]:
ds.DBZH.plot(cmap="HomeyerRainbow")

## backend_kwargs

Beside `first_dim` there are several additional backend_kwargs for the `uf` backend, which handle different aspects of angle alignment. This comes into play, when azimuth and/or elevation arrays are not evenly spacend and other issues.

In [ ]:
help(xd.io.UFBackendEntrypoint)

In [ ]:
ds = xr.open_dataset(fname, group="sweep_0", engine="uf", first_dim="time")
display(ds)

## open_uf_datatree

The same works analoguous with the datatree loader. But additionally we can provide a sweep string, number or list.

In [ ]:
help(xd.io.open_uf_datatree)

In [ ]:
dtree = xd.io.open_uf_datatree(fname, sweep=4)
display(dtree)

### Plot Sweep Range vs. Time

In [ ]:
dtree["sweep_4"].ds.DBZH.sortby("time").plot(y="time", cmap="HomeyerRainbow")

### Plot Sweep Range vs. Azimuth

In [ ]:
dtree["sweep_4"].ds.DBZH.plot(cmap="HomeyerRainbow")

In [ ]:
dtree = xd.io.open_uf_datatree(fname, sweep="sweep_8")
display(dtree)

In [ ]:
dtree = xd.io.open_uf_datatree(fname, sweep=[0, 1, 8])
display(dtree)

In [ ]:
dtree["sweep_0"]["sweep_fixed_angle"].values

In [ ]:
dtree["sweep_8"]["sweep_fixed_angle"].values

In [ ]:
dtree = xd.io.open_uf_datatree(fname)
display(dtree)

In [ ]:
dtree["sweep_1"]

## clean up

In [ ]:
tmpdir_obj.cleanup()